In [1]:
from datasets import load_dataset

ds = load_dataset("antoniomenezes/go_emotions_ptbr")

Using custom data configuration raw-30cf391cb2a99fe8
Found cached dataset csv (/users/jmperez/.cache/huggingface/datasets/antoniomenezes___csv/raw-30cf391cb2a99fe8/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

In [2]:
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'id', 'author', 'subreddit', 'link_id', 'parent_id', 'created_utc', 'rater_id', 'example_very_unclear', 'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral', 'texto'],
        num_rows: 211225
    })
})

In [3]:
df = ds["train"].to_pandas()



In [4]:
df["id"].value_counts()

eew5j0j    5
eeem310    5
ed9km8y    5
ed1djsl    5
ee8wvo4    5
          ..
ee0v8a8    1
eerolx8    1
ed7via6    1
eellksw    1
edcp8f5    1
Name: id, Length: 58011, dtype: int64

In [5]:

ids = df["id"].value_counts()

# Get ids with more than 3 ratings
ids = set(ids[ids >= 3].index)

In [6]:
columns = [ 
    'id',
    'example_very_unclear',
    'admiration',
    'amusement',
    'anger',
    'annoyance',
    'approval',
    'caring',
    'confusion',
    'curiosity',
    'desire',
    'disappointment',
    'disapproval',
    'disgust',
    'embarrassment',
    'excitement',
    'fear',
    'gratitude',
    'grief',
    'joy',
    'love',
    'nervousness',
    'optimism',
    'pride',
    'realization',
    'relief',
    'remorse',
    'sadness',
    'surprise',
    'neutral',
]

df_agg = df[columns].groupby("id") 
# Add a count column
df_agg = df_agg.sum()


df_agg

,example_very_unclear,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
id,,,,,,,,,,,,,,,,,,,,,
eczazk6,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
eczb07q,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
eczb4bm,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
eczb527,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
eczb6r7,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
efhclgb,0,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
efhcm78,0,0,0,0,2,1,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
efhcnhz,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3


In [7]:
df_agg.loc[ids]

/tmp/ipykernel_1834290/2230880486.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df_agg.loc[ids]


,example_very_unclear,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
id,,,,,,,,,,,,,,,,,,,,,
edwp8ma,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
efev8nq,0,0,0,0,0,0,1,0,0,0,...,0,0,2,0,2,0,0,0,0,0
ee1a7z1,0,0,2,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,1,1
edlve90,0,0,0,1,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
eepiz2e,0,0,0,0,0,0,0,2,1,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eedkt8w,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
ef7odkl,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,2,0
edsseyh,0,0,0,0,0,2,0,0,0,0,...,0,0,1,0,2,0,0,0,0,1


In [8]:
df_agg = df_agg[df_agg["example_very_unclear"] == 0]

df_agg

,example_very_unclear,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
id,,,,,,,,,,,,,,,,,,,,,
eczazk6,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
eczb07q,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
eczb4bm,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
eczb527,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
eczb6r7,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
efhclgb,0,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
efhcm78,0,0,0,0,2,1,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
efhcnhz,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3


In [9]:
emotions = columns[2:]

df_final = df_agg[emotions] >= 2

In [10]:
df_final.sum()

admiration         5031
amusement          2801
anger              1895
annoyance          2967
approval           3585
caring             1342
confusion          1598
curiosity          2637
desire              786
disappointment     1538
disapproval        2489
disgust             992
embarrassment       366
excitement         1023
fear                744
gratitude          3331
grief                95
joy                1748
love               2517
nervousness         204
optimism           1941
pride               136
realization        1351
relief              181
remorse             650
sadness            1572
surprise           1282
neutral           16747
dtype: int64

In [11]:
# Add the text columns!

df_final = df_final.join(df[["id", "text", "texto"]].set_index("id"))


In [12]:
df_final.rename(columns={"texto": "text_pt"}, inplace=True)

